# Regression Analysis of Small Data Using K-Fold

A Model for predicting house prices in Boston

## Preparation

### Load the Library

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

from utils.support_tf import LogLevelManager as llm
llm.set(2)

### Prepare Training Data

In [2]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.boston_housing.load_data()
type(train_x), train_x.shape, test_x.shape, type(test_y), test_y[0]

65536/57026 [==================================] - 0s 1us/step


(numpy.ndarray, (404, 13), (102, 13), numpy.ndarray, 7.2)